<a href="https://colab.research.google.com/github/sushirito/Molecular-Dynamics/blob/OpenMM/tleap_simplification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Install AmberTools
!pip install -q condacolab
import condacolab
condacolab.install()  # Installs Conda
!mamba install -c conda-forge ambertools -y

✨🍰✨ Everything looks OK!

Looking for: ['ambertools']

conda-forge/linux-64                                        Using cache
conda-forge/noarch                                          Using cache

Pinned packages:
  - python 3.11.*
  - python 3.11.*
  - python_abi 3.11.* *cp311*
  - cuda-version 12.*


Transaction

  Prefix: /usr/local

  All requested packages already installed



In [3]:
!pip install openmm
!pip install MDAnalysis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 84.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 72.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 96.3 MB/s eta 0:00:00


In [17]:
!cat $AMBERHOME/dat/leap/lib/atomic_ions.cmd

cat: /dat/leap/lib/atomic_ions.cmd: No such file or directory


In [18]:
!ls -la

total 92
drwxr-xr-x 1 root root  4096 Jan 27 05:14 .
drwxr-xr-x 1 root root  4096 Jan 27 04:34 ..
-rw-r--r-- 1 root root   277 Jan 27 05:04 ca_far_water.pdb
-rw-r--r-- 1 root root 21477 Jan 27 04:35 condacolab_install.log
drwxr-xr-x 4 root root  4096 Jan 23 14:23 .config
-rw-r--r-- 1 root root   276 Jan 27 04:59 hg_far_water.pdb
-rw-r--r-- 1 root root   891 Jan 27 04:37 hg.pdb
-rw-r--r-- 1 root root 22610 Jan 27 05:04 leap.log
drwxr-xr-x 1 root root  4096 Jan 23 14:24 sample_data
-rw-r--r-- 1 root root  1089 Jan 27 05:14 simulation_setup.log
-rw-r--r-- 1 root root   275 Jan 27 05:14 test.pdb
-rw-r--r-- 1 root root   694 Jan 27 05:14 tleap_CaCl2.in
-rw-r--r-- 1 root root   392 Jan 27 05:06 tleap.in


#Gem code

In [20]:
# Gem code

import subprocess

# Create the PDB content for Calcium with water molecules
pdb_file = """
ATOM      1  CA  CAL     1       0.000   0.000   0.000  1.00  0.00
TER
HETATM    2  OH2 HOH     2       5.000   5.000   5.000  1.00  0.00
HETATM    3  H1  HOH     2       5.816   5.000   5.221  1.00  0.00
HETATM    4  H2  HOH     2       4.184   5.000   5.221  1.00  0.00
TER
"""

# Save the PDB content to a file
with open('test.pdb', 'w') as f:
    f.write(pdb_file)

print("PDB file 'test.pdb' created successfully!")

# Prepare the TLEAP input file for Calcium
tleap_script = """
source leaprc.ff14SB
loadOff atomic_ions.lib #residue name for ions
loadamberparams frcmod.ions1lsm_1264_tip3p #monovalent ions
loadamberparams frcmod.ionslm_1264_tip3p #divalent ions
loadamberparams frcmod.ions34lsm_1264_tip3p #trivalent ions
i = loadpdb test.pdb
check i
solvatebox i TIP3PBOX 10 0.75
addions i CA 10 CL 20
charge i
saveamberparm i test_CaCl2.prmtop test_CaCl2.inpcrd
quit
"""

# Save the TLEAP script to a file
with open('tleap.in', 'w') as f:
    f.write(tleap_script)

print("TLEAP script 'tleap.in' created successfully!")

# Run TLEAP to generate PRMTOP and INPCRD
subprocess.run(['tleap', '-f', 'tleap.in'], check=True)
print("TLEAP executed successfully!")

# Modify the PRMTOP to include 12-6-4 LJ potential using parmed.py
parmed_commands = """
add12_6_4 :CA,CL
outparm modified_CaCl2.prmtop
quit
"""

process = subprocess.Popen(['parmed.py', 'test_CaCl2.prmtop'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
stdout, stderr = process.communicate(parmed_commands)

if process.returncode != 0:
    print("Error in parmed.py modification:", stderr)
else:
    print("PRMTOP modified to include 12-6-4 LJ potential successfully!")

# Prepare the SANDER input file (mdin)
mdin = """
&cntrl
    imin=0,
    ntx=1,
    irest=1,
    nstlim=5000,
    dt=0.002,
    ntc=2,
    ntf=2,
    lj1264=1,
    ntpr=100,
    ntwx=100,
    ntwr=100,
    /
"""

with open('mdin', 'w') as f:
    f.write(mdin)

print("SANDER input file 'mdin' created successfully!")

# Run SANDER
subprocess.run(['sander', '-O', '-i', 'mdin', '-p', 'modified_CaCl2.prmtop', '-c', 'test_CaCl2.inpcrd', '-o', 'mdout', '-r', 'mdrst', '-x', 'mdcrd'], check=True)
print("Simulation completed with SANDER. Outputs saved as 'mdcrd'.")

# Note: The OpenMM section has been removed as it is incompatible with the 12-6-4 LJ potential


PDB file 'test.pdb' created successfully!
TLEAP script 'tleap.in' created successfully!


CalledProcessError: Command '['tleap', '-f', 'tleap.in']' returned non-zero exit status 31.

In [ ]:
# Create the PDB content for Calcium with water molecules
pdb_file = """
ATOM      1  CA  CAL     1       0.000   0.000   0.000  1.00  0.00
TER
HETATM    2  OH2 HOH     2       5.000   5.000   5.000  1.00  0.00
HETATM    3  H1  HOH     2       5.816   5.000   5.221  1.00  0.00
HETATM    4  H2  HOH     2       4.184   5.000   5.221  1.00  0.00
TER
"""

# Save the PDB content to a file
with open('test.pdb', 'w') as f:
    f.write(pdb_file)

print("PDB file 'test.pdb' created successfully!")

In [19]:
# Prepare the exact TLEAP input file for Calcium
tleap_script = """
source leaprc.ff14SB
loadOff atomic_ions.lib #residue name for ions
loadamberparams frcmod.ions1lsm_1264_tip3p #monovalent ions
loadamberparams frcmod.ionslm_1264_tip3p #divalent ions
loadamberparams frcmod.ions34lsm_1264_tip3p #trivalent ions
i = loadpdb test.pdb
check i
solvatebox i TIP3PBOX 10 0.75
addions i CA 10 CL 20
charge i
saveamberparm i test_CaCl2.prmtop test_CaCl2.inpcrd
quit
"""

# Save the TLEAP script to a file
with open('tleap.in', 'w') as f:
    f.write(tleap_script)

print("TLEAP script 'tleap.in' created successfully!")

# Run TLEAP
!tleap -f tleap.in

# Generate trajectory using OpenMM
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PDBReporter, PME, HBonds
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import kelvin, picoseconds, nanometer

# Load Amber files
prmtop = AmberPrmtopFile('test_CaCl2.prmtop')
inpcrd = AmberInpcrdFile('test_CaCl2.inpcrd')

# Create system
system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picoseconds, 0.004*picoseconds)
platform = Platform.getPlatformByName('CPU')

# Initialize simulation
simulation = Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('catrajectory.pdb', 100))
simulation.step(5000)

print("Simulation completed. Trajectory saved as 'catrajectory.pdb'.")

TLEAP script 'tleap.in' created successfully!
-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in

/usr/local/bin/teLeap: Fatal Error!
Could not open file leaprc.ff14SB: not found

Exiting LEaP: Errors = 1; Warnings = 0; Notes = 0.


FileNotFoundError: [Errno 2] No such file or directory: 'test_CaCl2.prmtop'

In [16]:
#!/usr/bin/env python3
"""
End-to-End Script for Setting Up and Running a Calcium-Centered Simulation Using AMBER14 with 12-6-4 LJ Potential

This script performs the following steps:
1. Checks and installs necessary Python dependencies.
2. Verifies the availability of AMBER tools (tleap, sander).
3. Generates the PDB file for calcium with water molecules.
4. Prepares and runs the tleap script to generate AMBER parameter and coordinate files.
5. Modifies the PRMTOP file to include the 12-6-4 LJ potential using ParmEd.
6. Prepares and runs the SANDER simulation.
7. Provides feedback and logs throughout the process.

Ensure that AMBER is installed and the AMBERHOME environment variable is set before running this script.
"""

import os
import subprocess
import sys
import shutil
import logging
import platform

# ----------------------------
# Configuration and Setup
# ----------------------------

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s [%(levelname)s] %(message)s',
    handlers=[
        logging.StreamHandler(sys.stdout),
        logging.FileHandler("simulation_setup.log")
    ]
)

# Define required Python packages
REQUIRED_PACKAGES = [
    'parmed',
    'openmm',  # If OpenMM is needed later; currently not used in SANDER simulation
]

def install_python_packages():
    """
    Install required Python packages using pip.
    """
    logging.info("Installing required Python packages...")
    try:
        import pip
        for package in REQUIRED_PACKAGES:
            subprocess.check_call([sys.executable, '-m', 'pip', 'install', package])
        logging.info("Python packages installed successfully.")
    except Exception as e:
        logging.error(f"Failed to install Python packages: {e}")
        sys.exit(1)

def check_amber_tools():
    """
    Verify that AMBER tools (tleap, sander) are available in the PATH.
    """
    logging.info("Checking availability of AMBER tools (tleap, sander)...")
    amber_tools = ['tleap', 'sander', 'parmed']
    missing_tools = []
    for tool in amber_tools:
        if shutil.which(tool) is None:
            missing_tools.append(tool)
    if missing_tools:
        logging.error(f"The following AMBER tools are missing or not in PATH: {', '.join(missing_tools)}")
        logging.error("Please install AMBER and ensure that tleap and sander are accessible in your PATH.")
        sys.exit(1)
    logging.info("All required AMBER tools are available.")

def set_amber_home():
    """
    Ensure that the AMBERHOME environment variable is set.
    """
    if 'AMBERHOME' not in os.environ:
        # Attempt to set AMBERHOME based on common installation paths
        common_paths = [
            '/usr/local/amber20',  # Example path; modify as needed
            '/opt/amber20',
            os.path.expanduser('~/amber20'),
        ]
        for path in common_paths:
            if os.path.isdir(path):
                os.environ['AMBERHOME'] = path
                logging.info(f"Set AMBERHOME to {path}")
                return
        logging.warning("AMBERHOME environment variable is not set and could not be auto-detected.")
    else:
        logging.info(f"AMBERHOME is set to {os.environ['AMBERHOME']}")

# ----------------------------
# Step 1: Install Dependencies and Check AMBER
# ----------------------------

def setup_environment():
    """
    Set up the environment by installing Python packages and checking AMBER tools.
    """
    install_python_packages()
    set_amber_home()
    check_amber_tools()

# ----------------------------
# Step 2: Generate the PDB File
# ----------------------------

def generate_pdb():
    """
    Generate the PDB file for Calcium with water molecules.
    """
    pdb_content = """
ATOM      1  CA  CAL     1       0.000   0.000   0.000  1.00  0.00
TER
HETATM    2  OH2 HOH     2       5.000   5.000   5.000  1.00  0.00
HETATM    3  H1  HOH     2       5.816   5.000   5.221  1.00  0.00
HETATM    4  H2  HOH     2       4.184   5.000   5.221  1.00  0.00
TER
"""
    pdb_filename = 'test.pdb'
    try:
        with open(pdb_filename, 'w') as f:
            f.write(pdb_content.strip())
        logging.info(f"PDB file '{pdb_filename}' created successfully!")
    except Exception as e:
        logging.error(f"Failed to create PDB file: {e}")
        sys.exit(1)

# ----------------------------
# Step 3: Prepare and Run TLeap
# ----------------------------

def prepare_tleap_script():
    """
    Prepare the TLeap input script for calcium and chloride ions.
    """
    tleap_script = """
source leaprc.ff14SB
loadOff $AMBERHOME/dat/leap/lib/atomic_ions.lib  # Residue name for ions
loadamberparams $AMBERHOME/dat/leap/parm/frcmod.ions1lsm_1264_tip3p  # Monovalent ions
loadamberparams $AMBERHOME/dat/leap/parm/frcmod.ionslm_1264_tip3p   # Divalent ions
loadamberparams $AMBERHOME/dat/leap/parm/frcmod.ions34lsm_1264_tip3p  # Trivalent ions

# Load the generated PDB file
mol = loadpdb test.pdb
check mol

# Solvate the system with TIP3P water model
solvatebox mol TIP3PBOX 10.0 0.75

# Add ions: 10 Ca2+ and 20 Cl-
addions mol CA 10
addions mol CL 20

# Assign charges
charge mol

# Save the parameter and coordinate files
saveamberparm mol test_CaCl2.prmtop test_CaCl2.inpcrd

quit
"""
    tleap_input_filename = 'tleap_CaCl2.in'
    try:
        with open(tleap_input_filename, 'w') as f:
            f.write(tleap_script.strip())
        logging.info(f"TLeap input script '{tleap_input_filename}' created successfully!")
    except Exception as e:
        logging.error(f"Failed to create TLeap input script: {e}")
        sys.exit(1)
    return tleap_input_filename

def run_tleap(tleap_input_filename):
    """
    Run TLeap with the provided input script.
    """
    logging.info("Running TLeap to generate PRMTOP and INPCRD files...")
    try:
        subprocess.run(['tleap', '-f', tleap_input_filename], check=True)
        logging.info("TLeap executed successfully!")
    except subprocess.CalledProcessError as e:
        logging.error(f"Error running TLeap: {e}")
        sys.exit(1)

# ----------------------------
# Step 4: Modify PRMTOP with ParmEd for 12-6-4 LJ Potential
# ----------------------------

def modify_prmtop():
    """
    Modify the PRMTOP file to include the 12-6-4 LJ potential using ParmEd.
    """
    original_prmtop = 'test_CaCl2.prmtop'
    modified_prmtop = 'modified_CaCl2.prmtop'

    if not os.path.isfile(original_prmtop):
        logging.error(f"Original PRMTOP file '{original_prmtop}' not found.")
        sys.exit(1)

    # Create ParmEd commands
    parmed_commands = """
add12_6_4 :CA,CL
outparm {}
quit
""".format(modified_prmtop)

    parmed_input_filename = 'parmed_commands.in'
    try:
        with open(parmed_input_filename, 'w') as f:
            f.write(parmed_commands.strip())
        logging.info(f"ParmEd input script '{parmed_input_filename}' created successfully!")
    except Exception as e:
        logging.error(f"Failed to create ParmEd input script: {e}")
        sys.exit(1)

    # Run ParmEd
    logging.info("Modifying PRMTOP file to include 12-6-4 LJ potential using ParmEd...")
    try:
        # Using subprocess with input redirection
        with open(parmed_input_filename, 'r') as infile:
            subprocess.run(['parmed', original_prmtop], stdin=infile, check=True)
        logging.info(f"PRMTOP file modified and saved as '{modified_prmtop}'!")
    except subprocess.CalledProcessError as e:
        logging.error(f"Error running ParmEd: {e}")
        sys.exit(1)
    except Exception as e:
        logging.error(f"Unexpected error during ParmEd execution: {e}")
        sys.exit(1)

# ----------------------------
# Step 5: Prepare and Run SANDER Simulation
# ----------------------------

def prepare_sander_input():
    """
    Prepare the SANDER input file with lj1264=1.
    """
    sander_input_content = """
&cntrl
  imin=0,
  irest=1,
  ntx=5,
  ntb=1,
  cut=10.0,
  ntc=2,
  ntf=2,
  igb=0,
  salt=0,
  nstlim=5000,
  dt=0.002,
  ntpr=100,
  ntwx=100,
  lj1264=1
/
"""
    sander_input_filename = 'sander_CaCl2.in'
    try:
        with open(sander_input_filename, 'w') as f:
            f.write(sander_input_content.strip())
        logging.info(f"SANDER input file '{sander_input_filename}' created successfully!")
    except Exception as e:
        logging.error(f"Failed to create SANDER input file: {e}")
        sys.exit(1)
    return sander_input_filename

def run_sander(sander_input_filename):
    """
    Run SANDER simulation using the modified PRMTOP and SANDER input files.
    """
    modified_prmtop = 'modified_CaCl2.prmtop'
    inpcrd_file = 'test_CaCl2.inpcrd'
    sander_output = 'sander_CaCl2.out'
    sander_restart = 'sander_CaCl2.rst'

    # Check if necessary files exist
    for file in [modified_prmtop, inpcrd_file, sander_input_filename]:
        if not os.path.isfile(file):
            logging.error(f"Required file '{file}' not found for SANDER simulation.")
            sys.exit(1)

    logging.info("Running SANDER simulation...")
    try:
        subprocess.run([
            'sander',
            '-O',
            '-i', sander_input_filename,
            '-p', modified_prmtop,
            '-c', inpcrd_file,
            '-r', sander_restart,
            '-o', sander_output
        ], check=True)
        logging.info(f"SANDER simulation completed successfully! Output saved to '{sander_output}'.")
    except subprocess.CalledProcessError as e:
        logging.error(f"Error running SANDER: {e}")
        sys.exit(1)

# ----------------------------
# Main Execution Flow
# ----------------------------

def main():
    """
    Main function to execute all steps.
    """
    logging.info("=== Starting Calcium-Centered Simulation Setup ===")

    # Step 1: Environment Setup
    setup_environment()

    # Step 2: Generate PDB File
    generate_pdb()

    # Step 3: Prepare and Run TLeap
    tleap_input = prepare_tleap_script()
    run_tleap(tleap_input)

    # Step 4: Modify PRMTOP with ParmEd
    modify_prmtop()

    # Step 5: Prepare and Run SANDER Simulation
    sander_input = prepare_sander_input()
    run_sander(sander_input)

    logging.info("=== All Steps Completed Successfully! ===")

if __name__ == "__main__":
    main()


2025-01-27 05:13:52,110 [INFO] === Starting Calcium-Centered Simulation Setup ===
2025-01-27 05:13:52,113 [INFO] Installing required Python packages...


/usr/local/lib/python3.11/dist-packages/_distutils_hack/__init__.py:31: UserWarning: Setuptools is replacing distutils. Support for replacing an already imported distutils is deprecated. In the future, this condition will fail. Register concerns at https://github.com/pypa/setuptools/issues/new?template=distutils-deprecation.yml
  warnings.warn(


2025-01-27 05:14:11,044 [INFO] Python packages installed successfully.
2025-01-27 05:14:11,046 [WARNING] AMBERHOME environment variable is not set and could not be auto-detected.
2025-01-27 05:14:11,050 [INFO] Checking availability of AMBER tools (tleap, sander)...
2025-01-27 05:14:11,051 [INFO] All required AMBER tools are available.
2025-01-27 05:14:11,053 [INFO] PDB file 'test.pdb' created successfully!
2025-01-27 05:14:11,054 [INFO] TLeap input script 'tleap_CaCl2.in' created successfully!
2025-01-27 05:14:11,055 [INFO] Running TLeap to generate PRMTOP and INPCRD files...
2025-01-27 05:14:11,076 [ERROR] Error running TLeap: Command '['tleap', '-f', 'tleap_CaCl2.in']' returned non-zero exit status 31.
2025-01-27 05:14:11,079 [ERROR] Internal Python error in the inspect module.
Below is the traceback from this internal error.

Traceback (most recent call last):
  File "<ipython-input-16-10a58360c512>", line 181, in run_tleap
    subprocess.run(['tleap', '-f', tleap_input_filename], c

TypeError: object of type 'NoneType' has no len()

In [14]:
# Create the PDB content for Calcium with water molecules
pdb_file = """
ATOM      1  CA  CAL     1       0.000   0.000   0.000  1.00  0.00
TER
HETATM    2  OH2 HOH     2       5.000   5.000   5.000  1.00  0.00
HETATM    3  H1  HOH     2       5.816   5.000   5.221  1.00  0.00
HETATM    4  H2  HOH     2       4.184   5.000   5.221  1.00  0.00
TER
"""

# Save the PDB content to a file
with open('test.pdb', 'w') as f:
    f.write(pdb_file)

print("PDB file 'test.pdb' created successfully!")

PDB file 'test.pdb' created successfully!


In [15]:
# Prepare the exact TLEAP input file for Calcium
tleap_script = """
source leaprc.ff14SB
loadOff atomic_ions.lib #residue name for ions
loadamberparams frcmod.ions1lsm_1264_tip3p #monovalent ions
loadamberparams frcmod.ionslm_1264_tip3p #divalent ions
loadamberparams frcmod.ions34lsm_1264_tip3p #trivalent ions
i = loadpdb test.pdb
check i
solvatebox i TIP3PBOX 10 0.75
addions i CA 10 CL 20
charge i
saveamberparm i test_CaCl2.prmtop test_CaCl2.inpcrd
quit
"""

# Save the TLEAP script to a file
with open('tleap.in', 'w') as f:
    f.write(tleap_script)

print("TLEAP script 'tleap.in' created successfully!")

# Run TLEAP
!tleap -f tleap.in

# Generate trajectory using OpenMM
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PDBReporter, PME, HBonds
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import kelvin, picoseconds, nanometer

# Load Amber files
prmtop = AmberPrmtopFile('test_CaCl2.prmtop')
inpcrd = AmberInpcrdFile('test_CaCl2.inpcrd')

# Create system
system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picoseconds, 0.004*picoseconds)
platform = Platform.getPlatformByName('CPU')

# Initialize simulation
simulation = Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('catrajectory.pdb', 100))
simulation.step(5000)

print("Simulation completed. Trajectory saved as 'catrajectory.pdb'.")

TLEAP script 'tleap.in' created successfully!
-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in

/usr/local/bin/teLeap: Fatal Error!
Could not open file leaprc.ff14SB: not found

Exiting LEaP: Errors = 1; Warnings = 0; Notes = 0.


FileNotFoundError: [Errno 2] No such file or directory: 'test_CaCl2.prmtop'

In [8]:
# Create the PDB content with the water molecule farther from Hg2+
pdb_file = """
ATOM      1  HG  HG     1       0.000   0.000   0.000  1.00  0.00
TER
HETATM    2  OH2 HOH     2       5.000   5.000   5.000  1.00  0.00
HETATM    3  H1  HOH     2       5.816   5.000   5.221  1.00  0.00
HETATM    4  H2  HOH     2       4.184   5.000   5.221  1.00  0.00
TER
"""

# Save the PDB content to a file
with open('hg_far_water.pdb', 'w') as f:
    f.write(pdb_file)

print("PDB file 'hg_far_water.pdb' created successfully!")

PDB file 'hg_far_water.pdb' created successfully!


In [10]:
# Prepare the tleap input file
tleap_script = """
source leaprc.protein.ff14SB
loadamberparams frcmod.ionsjc_tip3p
loadamberparams frcmod.ionslm_1264_tip3p
loadamberparams frcmod.ions34lsm_1264_tip3p
mol = loadpdb hg_far_water.pdb
solvatebox mol TIP3PBOX 10 0.75
addions mol CA 10 CL 20
charge mol
saveamberparm mol test_CaCl2.prmtop test_CaCl2.inpcrd
quit
"""


# Save tleap script
with open('tleap.in', 'w') as f:
    f.write(tleap_script)

# Run tleap
!tleap -f tleap.in

# Generate trajectory using OpenMM
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PDBReporter, PME, HBonds
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import kelvin, picoseconds, nanometer

# Load Amber files
prmtop = AmberPrmtopFile('solv.prmtop')
inpcrd = AmberInpcrdFile('solv.inpcrd')

# Create system
system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picoseconds, 0.004*picoseconds)
platform = Platform.getPlatformByName('CPU')

# Initialize simulation
simulation = Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('hgtrajectory.pdb', 100))
simulation.step(5000)

print("Simulation completed. Trajectory saved as 'hgtrajectory.pdb'.")


-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /usr/local/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /usr/local/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /usr/local/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /usr/local/dat/leap/lib/amino12.lib
Loading library: /usr/local/dat/leap/lib/aminoct12.lib
Loading library: /usr/local/dat/leap/lib/aminont12.lib
Loading parameters: /usr/local/dat/leap/parm/frcmod.ionsjc_tip3p
Reading force field modificati

FileNotFoundError: [Errno 2] No such file or directory: 'solv.prmtop'

#Other code

In [ ]:
# Prepare the tleap input file
tleap_script = """
source leaprc.protein.ff14SB
source leaprc.water.tip3p
loadAmberParams frcmod.ions234lm_126_tip3p

# Create an empty unit named 'solv'
solv = createUnit solv

# Fill 'solv' with TIP3P water (10.0 Å buffer)
solvateBox solv TIP3PBOX 10.0

# Add one Hg2+ and one Cl-
addIons2 solv Hg2+ 1
addIons2 solv Cl- 1

# Save Amber topology and coordinate files
saveAmberParm solv solv.prmtop solv.inpcrd
quit
"""


# Save tleap script
with open('tleap.in', 'w') as f:
    f.write(tleap_script)

# Run tleap
!tleap -f tleap.in

# Generate trajectory using OpenMM
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PDBReporter, PME, HBonds
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import kelvin, picoseconds, nanometer

# Load Amber files
prmtop = AmberPrmtopFile('solv.prmtop')
inpcrd = AmberInpcrdFile('solv.inpcrd')

# Create system
system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picoseconds, 0.004*picoseconds)
platform = Platform.getPlatformByName('CPU')

# Initialize simulation
simulation = Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('hgtrajectory.pdb', 100))
simulation.step(5000)

print("Simulation completed. Trajectory saved as 'hgtrajectory.pdb'.")


-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /usr/local/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /usr/local/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /usr/local/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /usr/local/dat/leap/lib/amino12.lib
Loading library: /usr/local/dat/leap/lib/aminoct12.lib
Loading library: /usr/local/dat/leap/lib/aminont12.lib
----- Source: /usr/local/dat/leap/cmd/leaprc.water.tip3p
----- Source of /usr/local/dat/leap/cm

FileNotFoundError: [Errno 2] No such file or directory: 'solv.prmtop'

In [ ]:
# Prepare the tleap input file
tleap_script = """
source leaprc.protein.ff14SB
source leaprc.water.tip3p
loadAmberParams frcmod.ions234lm_126_tip3p

# Create an empty unit named 'solv'
solv = createUnit solv

# Fill 'solv' with TIP3P water (10.0 Å buffer)
solvateBox solv TIP3PBOX 10.0

# Add one Hg2+ and one Cl-
addIons2 solv Hg2+ 1

# Save Amber topology and coordinate files
saveAmberParm solv solv.prmtop solv.inpcrd
quit
"""

# Save tleap script
with open('tleap.in', 'w') as f:
    f.write(tleap_script)

# Run tleap
!tleap -f tleap.in

# Locate .prmtop and .inpcrd files
prmtop_path = os.path.abspath('solv.prmtop')
inpcrd_path = os.path.abspath('solv.inpcrd')

print(f"Located Amber topology file: {prmtop_path}")
print(f"Located Amber coordinate file: {inpcrd_path}")

# Generate trajectory using OpenMM
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PDBReporter, PME, HBonds
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import kelvin, picoseconds, nanometer

# Load Amber files
prmtop = AmberPrmtopFile(prmtop_path)
inpcrd = AmberInpcrdFile(inpcrd_path)

# Create system
system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picoseconds, 0.004*picoseconds)
platform = Platform.getPlatformByName('CPU')

# Initialize simulation
simulation = Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('hgtrajectory.pdb', 100))
simulation.step(5000)

print("Simulation completed. Trajectory saved as 'hgtrajectory.pdb'.")


-I: Adding /usr/local/dat/leap/prep to search path.
-I: Adding /usr/local/dat/leap/lib to search path.
-I: Adding /usr/local/dat/leap/parm to search path.
-I: Adding /usr/local/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /usr/local/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /usr/local/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /usr/local/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /usr/local/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidechain parameters
Loading library: /usr/local/dat/leap/lib/amino12.lib
Loading library: /usr/local/dat/leap/lib/aminoct12.lib
Loading library: /usr/local/dat/leap/lib/aminont12.lib
----- Source: /usr/local/dat/leap/cmd/leaprc.water.tip3p
----- Source of /usr/local/dat/leap/cm

FileNotFoundError: [Errno 2] No such file or directory: '/content/solv.prmtop'

In [ ]:
import subprocess
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PDBReporter, PME, HBonds
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import kelvin, picoseconds, nanometer

# Create a minimal structure to solvate
dummy_pdb = """\
ATOM      1  O   HOH A   1       0.000   0.000   0.000  1.00  0.00
END
"""
with open("dummy.pdb", "w") as f:
    f.write(dummy_pdb)

# Corrected tleap input script
tleap_script = """\
source leaprc.protein.ff14SB
source leaprc.water.tip3p
loadAmberParams frcmod.ions234lm_126_tip3p

mol = loadPdb dummy.pdb
solv = solvateBox mol TIP3PBOX 10.0

addIons2 solv Hg2+ 1
addIons2 solv Cl-  1

saveAmberParm solv solv.prmtop solv.inpcrd
quit
"""
with open('tleap.in', 'w') as f:
    f.write(tleap_script)

# Run tleap and capture errors
try:
    result = subprocess.run(['tleap', '-f', 'tleap.in'], check=True, capture_output=True, text=True)
except subprocess.CalledProcessError as e:
    print("TLeap failed:")
    print(e.stderr)
    raise

# Continue with simulation if tleap succeeded
prmtop = AmberPrmtopFile('solv.prmtop')
inpcrd = AmberInpcrdFile('solv.inpcrd')

system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picoseconds, 0.004*picoseconds)
platform = Platform.getPlatformByName('CPU')

simulation = Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('hgtrajectory.pdb', 100))
simulation.step(5000)

print("Simulation completed. Trajectory saved as 'hgtrajectory.pdb'.")


TLeap failed:



CalledProcessError: Command '['tleap', '-f', 'tleap.in']' returned non-zero exit status 21.

In [ ]:
import subprocess
from openmm.app import AmberPrmtopFile, AmberInpcrdFile, Simulation, PDBReporter, PME, HBonds
from openmm import LangevinMiddleIntegrator, Platform
from openmm.unit import kelvin, picoseconds, nanometer

# Create a minimal structure to solvate
dummy_pdb = """\
ATOM      1  O   HOH A   1       0.000   0.000   0.000  1.00  0.00
END
"""
with open("dummy.pdb", "w") as f:
    f.write(dummy_pdb)

# Corrected tleap input script
tleap_script = """\

source leaprc.protein.ff14SB
source leaprc.water.tip3p
source atomic_ions.lib

loadAmberParams frcmod.ions234lm_126_tip3p

molecule = loadPdb dummy.pdb

solvateBox molecule TIP3PBOX 10.0

addIons molecule Hg 0

saveAmberParm molecule molecule.prmtop molecule.inpcrd

savePdb molecule molecule_solvated.pdb

quit

"""
with open('tleap.in', 'w') as f:
    f.write(tleap_script)

# Run tleap and capture errors
try:
    result = subprocess.run(['tleap', '-f', 'tleap.in'], check=True, capture_output=True, text=True)
except subprocess.CalledProcessError as e:
    print("TLeap failed:")
    print(e.stderr)
    raise

# Continue with simulation if tleap succeeded
prmtop = AmberPrmtopFile('solv.prmtop')
inpcrd = AmberInpcrdFile('solv.inpcrd')

system = prmtop.createSystem(nonbondedMethod=PME, nonbondedCutoff=1.0*nanometer, constraints=HBonds)
integrator = LangevinMiddleIntegrator(300*kelvin, 1/picoseconds, 0.004*picoseconds)
platform = Platform.getPlatformByName('CPU')

simulation = Simulation(prmtop.topology, system, integrator, platform)
simulation.context.setPositions(inpcrd.positions)
simulation.minimizeEnergy()
simulation.reporters.append(PDBReporter('hgtrajectory.pdb', 100))
simulation.step(5000)

print("Simulation completed. Trajectory saved as 'hgtrajectory.pdb'.")

TLeap failed:



CalledProcessError: Command '['tleap', '-f', 'tleap.in']' returned non-zero exit status 31.